In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import Dense, SimpleRNN, BatchNormalization
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import accuracy_score
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import nltk
import re
from nltk.tokenize import word_tokenize
import string
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/goodreads-books-reviews-290312/goodreads_test.csv
/kaggle/input/goodreads-books-reviews-290312/goodreads_sample_submission.csv
/kaggle/input/goodreads-books-reviews-290312/goodreads_train.csv


## Reading and checking dataset

In [3]:
df = pd.read_csv('../input/goodreads-books-reviews-290312/goodreads_train.csv')
df.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,NaN,NaN,1,0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,NaN,NaN,5,1
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1


In [4]:
df.isna().sum()

user_id              0
book_id              0
review_id            0
rating               0
review_text          0
date_added           0
date_updated         0
read_at          91766
started_at      274297
n_votes              0
n_comments           0
dtype: int64

In [5]:
# Taking only the columns that interest us. The review and ratings.
df = df[["rating", "review_text"]]
df.head()

,rating,review_text
0,5,This is a special book. It started slow for ab...
1,3,Recommended by Don Katz. Avail for free in Dec...
2,3,"A fun, fast paced science fiction thriller. I ..."
3,0,Recommended reading to understand what is goin...
4,4,"I really enjoyed this book, and there is a lot..."


In [6]:
df['rating'].value_counts()

4    313688
5    265007
3    188972
2     72627
0     30988
1     28718
Name: rating, dtype: int64

## Balancing our dataset

I took the amount from the review with less quantity, then I made all the other have the same amount, that is udersampling methode for balancing data .

In [7]:
df = df.groupby('rating').head(28718)
df.head()

,rating,review_text
0,5,This is a special book. It started slow for ab...
1,3,Recommended by Don Katz. Avail for free in Dec...
2,3,"A fun, fast paced science fiction thriller. I ..."
3,0,Recommended reading to understand what is goin...
4,4,"I really enjoyed this book, and there is a lot..."


In [8]:
df['rating'].value_counts()

5    28718
3    28718
0    28718
4    28718
2    28718
1    28718
Name: rating, dtype: int64

## Data Processing


Now we cleaned the reviews from symbols, HTML, URL, punctuation, numbers, emojis and capital letter.  All things that do not influence our interpretation of the data for sentiment analysis

In [9]:
def clean_data(data):
    #Remove HTML
    data = re.sub(r'<.*?>', '',  data)
    #Remove URL
    data = re.sub(r'http\S+', '', data)
    #Remove punctuation
    data = re.sub(r'[^\w\s]', '', data)
    #Remove numbers
    data = re.sub(r'\b\d+\b', '', data)
    #Create a list os emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        "]+", flags=re.UNICODE)
    #Remove emojis
    data = emoji_pattern.sub(r'', data)
    #Make all letters lower case
    data = data.lower()
    return data

In [10]:
df['review_text'] = df['review_text'].apply(lambda z: clean_data(z))
df.head()

,rating,review_text
0,5,this is a special book it started slow for abo...
1,3,recommended by don katz avail for free in dece...
2,3,a fun fast paced science fiction thriller i re...
3,0,recommended reading to understand what is goin...
4,4,i really enjoyed this book and there is a lot ...


In [11]:
# Now i split each review from a big string to a vector of strings for each word.
df['review_text'] = df.apply(lambda row: nltk.word_tokenize(row['review_text']), axis=1)
df.head()

,rating,review_text
0,5,"[this, is, a, special, book, it, started, slow..."
1,3,"[recommended, by, don, katz, avail, for, free,..."
2,3,"[a, fun, fast, paced, science, fiction, thrill..."
3,0,"[recommended, reading, to, understand, what, i..."
4,4,"[i, really, enjoyed, this, book, and, there, i..."


In [12]:
# Now i removed all the stopwords from the english language

stop = stopwords.words('english')

df['review_text'] = df['review_text'].apply(lambda x: [word for word in x if word not in (stop)])
df.head()

,rating,review_text
0,5,"[special, book, started, slow, first, third, m..."
1,3,"[recommended, katz, avail, free, december]"
2,3,"[fun, fast, paced, science, fiction, thriller,..."
3,0,"[recommended, reading, understand, going, midd..."
4,4,"[really, enjoyed, book, lot, recommend, drag, ..."


### Lemmatization


Lemmatization is basically replacing different forms of the same word for a single one. 
Ex:
    words = ["connects", "connected", "strange", "am", "is"]
    lemmatized = ["connect", "connect", "strange", "be", "be"]



In [13]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [14]:
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

from nltk.corpus import wordnet

def lemma_traincorpus(data):
    lemmatizer=WordNetLemmatizer()
    out_data= []
    for words in data:
        out_data.append(lemmatizer.lemmatize(words, get_wordnet_pos(words)))
    return out_data

In [15]:
df['review_text']=df['review_text'].apply(lambda z: lemma_traincorpus(z))
df.head()

,rating,review_text
0,5,"[special, book, start, slow, first, third, mid..."
1,3,"[recommend, katz, avail, free, december]"
2,3,"[fun, fast, pace, science, fiction, thriller, ..."
3,0,"[recommend, reading, understand, go, middle, a..."
4,4,"[really, enjoy, book, lot, recommend, drag, li..."


In [18]:
df.head()

,rating,review_text
0,5,"[special, book, start, slow, first, third, mid..."
1,3,"[recommend, katz, avail, free, december]"
2,3,"[fun, fast, pace, science, fiction, thriller, ..."
3,0,"[recommend, reading, understand, go, middle, a..."
4,4,"[really, enjoy, book, lot, recommend, drag, li..."


### TF IDF:

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
%%time
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words=stopwords,
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(df.review_text)
train_word_features = word_vectorizer.transform(train_text)